# Machine Learning Specialization - Unsupervised Learning
## Week 1: Clustering and Anomaly Detection

### Learning Objectives:
- Understand unsupervised learning and its applications
- Implement K-means clustering algorithm from scratch
- Apply clustering to real datasets and evaluate results
- Learn anomaly detection using statistical methods
- Implement Gaussian anomaly detection algorithm

### Key Concepts:
- **Unsupervised Learning**: Learning from unlabeled data
- **Clustering**: Grouping similar data points together
- **K-means**: Popular clustering algorithm using centroids
- **Anomaly Detection**: Finding unusual or outlier data points
- **Gaussian Distribution**: Statistical model for anomaly detection

Unlike supervised learning, unsupervised methods work without labeled training examples.

### 1. Import Required Libraries

Let's import the necessary libraries for our clustering and anomaly detection exercises.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_classification
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
from scipy.stats import multivariate_normal
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

### 2. Generate Clustering Dataset

We'll create synthetic datasets to demonstrate clustering algorithms.

In [ ]:
# Generate synthetic clustering data
X_clusters, y_true_clusters = make_blobs(
    n_samples=300, 
    centers=4, 
    cluster_std=1.5, 
    random_state=42
)

# Generate data with different cluster shapes
X_moons, y_moons = make_classification(
    n_samples=300, 
    n_features=2, 
    n_informative=2, 
    n_redundant=0, 
    n_clusters_per_class=1, 
    class_sep=2,
    random_state=42
)

print(f"Clusters dataset: X = {X_clusters.shape}")
print(f"Moons dataset: X = {X_moons.shape}")
print(f"True cluster labels (clusters): {np.unique(y_true_clusters)}")

# Visualize datasets
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_clusters[:, 0], X_clusters[:, 1], c=y_true_clusters, cmap='viridis', alpha=0.7)
plt.title('Well-Separated Clusters')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='True Cluster')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis', alpha=0.7)
plt.title('Non-Spherical Clusters')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Class')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3. K-means Clustering Algorithm

K-means is an iterative algorithm that partitions data into K clusters:

1. **Initialize**: Randomly choose K cluster centroids
2. **Assign**: Assign each point to the nearest centroid
3. **Update**: Move centroids to the mean of their assigned points
4. **Repeat**: Until convergence or max iterations

The objective is to minimize the within-cluster sum of squares (WCSS).

In [ ]:
def kmeans_from_scratch(X, K, max_iters=100, tol=1e-4):
    """
    Implement K-means clustering from scratch.
    
    Args:
        X: Data points (n_samples x n_features)
        K: Number of clusters
        max_iters: Maximum iterations
        tol: Tolerance for convergence
    
    Returns:
        centroids: Final cluster centroids
        labels: Cluster assignments for each point
        wcss_history: Within-cluster sum of squares over iterations
    """
    n_samples, n_features = X.shape
    
    # Initialize centroids randomly
    centroids = X[np.random.choice(n_samples, K, replace=False)]
    
    wcss_history = []
    
    for iteration in range(max_iters):
        # Assign each point to nearest centroid
        distances = np.zeros((n_samples, K))
        for k in range(K):
            distances[:, k] = np.sum((X - centroids[k]) ** 2, axis=1)
        
        labels = np.argmin(distances, axis=1)
        
        # Calculate WCSS
        wcss = 0
        for k in range(K):
            cluster_points = X[labels == k]
            if len(cluster_points) > 0:
                wcss += np.sum((cluster_points - centroids[k]) ** 2)
        wcss_history.append(wcss)
        
        # Update centroids
        new_centroids = np.zeros_like(centroids)
        for k in range(K):
            cluster_points = X[labels == k]
            if len(cluster_points) > 0:
                new_centroids[k] = np.mean(cluster_points, axis=0)
        
        # Check convergence
        if np.allclose(centroids, new_centroids, atol=tol):
            break
        
        centroids = new_centroids
        
        if iteration % 10 == 0:
            print(f"Iteration {iteration}: WCSS = {wcss:.4f}")
    
    return centroids, labels, wcss_history

# Test K-means on our data
K = 4
centroids, labels, wcss_history = kmeans_from_scratch(X_clusters, K)

print(f"\nFinal WCSS: {wcss_history[-1]:.4f}")
print(f"Centroids:\n{centroids}")

# Visualize K-means results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_clusters[:, 0], X_clusters[:, 1], c=labels, cmap='viridis', alpha=0.7)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=200, linewidth=3, label='Centroids')
plt.title('K-means Clustering Results')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(wcss_history, 'b-o')
plt.xlabel('Iteration')
plt.ylabel('Within-Cluster Sum of Squares')
plt.title('K-means Convergence')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4. Choosing K: The Elbow Method

How do we choose the optimal number of clusters K?

- **Elbow Method**: Plot WCSS vs K, look for the "elbow" point
- **Silhouette Score**: Measures how similar points are to their own cluster vs other clusters
- **Gap Statistic**: Compares within-cluster dispersion to reference distribution

In [ ]:
def evaluate_k_choice(X, max_K=10):
    """Evaluate different values of K using multiple metrics"""
    
    wcss_values = []
    silhouette_scores = []
    calinski_scores = []
    
    K_range = range(2, max_K + 1)
    
    for K in K_range:
        # Run K-means
        kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        
        # Calculate metrics
        wcss = kmeans.inertia_
        silhouette = silhouette_score(X, labels)
        calinski = calinski_harabasz_score(X, labels)
        
        wcss_values.append(wcss)
        silhouette_scores.append(silhouette)
        calinski_scores.append(calinski)
        
        print(f"K = {K}: WCSS = {wcss:.2f}, Silhouette = {silhouette:.3f}, Calinski = {calinski:.2f}")
    
    return K_range, wcss_values, silhouette_scores, calinski_scores

# Evaluate K for our dataset
K_range, wcss_values, silhouette_scores, calinski_scores = evaluate_k_choice(X_clusters)

# Plot evaluation metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Elbow method
axes[0].plot(K_range, wcss_values, 'bo-')
axes[0].set_xlabel('Number of Clusters (K)')
axes[0].set_ylabel('Within-Cluster Sum of Squares')
axes[0].set_title('Elbow Method')
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='True K=4')
axes[0].legend()

# Silhouette score
axes[1].plot(K_range, silhouette_scores, 'go-')
axes[1].set_xlabel('Number of Clusters (K)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Analysis')
axes[1].grid(True, alpha=0.3)
axes[1].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='True K=4')
axes[1].legend()

# Calinski-Harabasz score
axes[2].plot(K_range, calinski_scores, 'ro-')
axes[2].set_xlabel('Number of Clusters (K)')
axes[2].set_ylabel('Calinski-Harabasz Score')
axes[2].set_title('Calinski-Harabasz Index')
axes[2].grid(True, alpha=0.3)
axes[2].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='True K=4')
axes[2].legend()

plt.tight_layout()
plt.show()

# Find optimal K
optimal_k_elbow = 4  # Visually from elbow plot
optimal_k_silhouette = K_range[np.argmax(silhouette_scores)]
optimal_k_calinski = K_range[np.argmax(calinski_scores)]

print(f"\nOptimal K suggestions:")
print(f"Elbow method: K = {optimal_k_elbow}")
print(f"Silhouette score: K = {optimal_k_silhouette}")
print(f"Calinski-Harabasz: K = {optimal_k_calinski}")
print(f"True K: 4")

### 5. Anomaly Detection Fundamentals

Anomaly detection identifies data points that deviate significantly from the norm:

- **Statistical Methods**: Model data as Gaussian distribution
- **Density-based**: Points in low-density regions are anomalies
- **Distance-based**: Points far from their neighbors are anomalies
- **Model-based**: Train models to distinguish normal from anomalous data

We'll focus on the statistical approach using Gaussian distributions.

### 6. Gaussian Anomaly Detection

The Gaussian approach models each feature as a Gaussian distribution:

$$p(x) = \prod_{j=1}^n p(x_j; \mu_j, \sigma_j^2) = \prod_{j=1}^n \frac{1}{\sqrt{2\pi} \sigma_j} \exp\left(-\frac{(x_j - \mu_j)^2}{2\sigma_j^2}\right)$$

Points with low probability (below threshold ε) are flagged as anomalies.

In [ ]:
def gaussian_anomaly_detection(X_train, X_val, epsilon=None):
    """
    Implement Gaussian anomaly detection.
    
    Args:
        X_train: Training data (normal examples)
        X_val: Validation data for threshold selection
        epsilon: Anomaly threshold (if None, will be selected)
    
    Returns:
        mu: Feature means
        sigma2: Feature variances
        epsilon: Selected threshold
    """
    # Estimate parameters from training data
    mu = np.mean(X_train, axis=0)
    sigma2 = np.var(X_train, axis=0, ddof=1)  # Use ddof=1 for unbiased estimate
    
    def multivariate_gaussian(X):
        """Compute multivariate Gaussian probability density"""
        n_features = X.shape[1]
        
        # Compute probability for each feature independently (diagonal covariance)
        probs = np.ones(X.shape[0])
        for j in range(n_features):
            probs *= (1 / np.sqrt(2 * np.pi * sigma2[j])) * \
                    np.exp(-((X[:, j] - mu[j]) ** 2) / (2 * sigma2[j]))
        
        return probs
    
    if epsilon is None:
        # Select threshold using validation set
        # Assume validation set has some anomalies (we'll simulate this)
        y_val = np.zeros(X_val.shape[0])  # Assume all normal for this demo
        y_val[:int(0.1 * len(y_val))] = 1  # Mark 10% as anomalies for demo
        
        # Compute probabilities on validation set
        p_val = multivariate_gaussian(X_val)
        
        # Try different thresholds
        best_epsilon = None
        best_f1 = 0
        
        for epsilon_candidate in np.logspace(-20, 0, 100):
            predictions = (p_val < epsilon_candidate).astype(int)
            
            # Calculate F1 score
            tp = np.sum((predictions == 1) & (y_val == 1))
            fp = np.sum((predictions == 1) & (y_val == 0))
            fn = np.sum((predictions == 0) & (y_val == 1))
            
            if tp + fp > 0 and tp + fn > 0:
                precision = tp / (tp + fp)
                recall = tp / (tp + fn)
                f1 = 2 * precision * recall / (precision + recall)
                
                if f1 > best_f1:
                    best_f1 = f1
                    best_epsilon = epsilon_candidate
        
        epsilon = best_epsilon
        print(f"Selected threshold ε = {epsilon:.2e}, F1 = {best_f1:.4f}")
    
    return mu, sigma2, epsilon

# Create anomaly detection dataset
# Normal data: multivariate Gaussian
np.random.seed(42)
n_normal = 1000
mu_normal = np.array([0, 0])
cov_normal = np.array([[1, 0.5], [0.5, 1]])
X_normal = np.random.multivariate_normal(mu_normal, cov_normal, n_normal)

# Anomalies: points far from the distribution
n_anomalies = 50
X_anomalies = np.random.uniform(-6, 6, (n_anomalies, 2))
# Keep only points that are actually anomalous
distances = np.sum((X_anomalies - mu_normal) ** 2, axis=1)
anomaly_mask = distances > 9  # Points more than 3 std away
X_anomalies = X_anomalies[anomaly_mask[:n_anomalies]]

# Combine datasets
X_anomaly_data = np.vstack([X_normal, X_anomalies])
y_anomaly_true = np.hstack([np.zeros(n_normal), np.ones(len(X_anomalies))])

print(f"Anomaly detection dataset: {X_anomaly_data.shape[0]} samples")
print(f"Normal samples: {np.sum(y_anomaly_true == 0)}")
print(f"Anomalies: {np.sum(y_anomaly_true == 1)}")

# Split for training and testing
X_train_anomaly = X_normal[:800]  # Only normal data for training
X_test_anomaly = X_anomaly_data[800:]
y_test_anomaly = y_anomaly_true[800:]

# Train anomaly detection model
mu, sigma2, epsilon = gaussian_anomaly_detection(X_train_anomaly, X_test_anomaly)

# Make predictions
def predict_anomalies(X, mu, sigma2, epsilon):
    """Predict anomalies using trained Gaussian model"""
    n_features = X.shape[1]
    probs = np.ones(X.shape[0])
    
    for j in range(n_features):
        probs *= (1 / np.sqrt(2 * np.pi * sigma2[j])) * \
                np.exp(-((X[:, j] - mu[j]) ** 2) / (2 * sigma2[j]))
    
    return (probs < epsilon).astype(int)

y_pred_anomaly = predict_anomalies(X_test_anomaly, mu, sigma2, epsilon)

# Evaluate
from sklearn.metrics import classification_report, confusion_matrix

print("\nAnomaly Detection Results:")
print(classification_report(y_test_anomaly, y_pred_anomaly, target_names=['Normal', 'Anomaly']))

# Visualize results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X_test_anomaly[y_test_anomaly == 0, 0], X_test_anomaly[y_test_anomaly == 0, 1], 
           c='blue', alpha=0.6, label='True Normal')
plt.scatter(X_test_anomaly[y_test_anomaly == 1, 0], X_test_anomaly[y_test_anomaly == 1, 1], 
           c='red', alpha=0.6, label='True Anomaly')
plt.title('True Labels')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_test_anomaly[y_pred_anomaly == 0, 0], X_test_anomaly[y_pred_anomaly == 0, 1], 
           c='blue', alpha=0.6, label='Predicted Normal')
plt.scatter(X_test_anomaly[y_pred_anomaly == 1, 0], X_test_anomaly[y_pred_anomaly == 1, 1], 
           c='red', alpha=0.6, label='Predicted Anomaly')
plt.title('Predictions')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show learned Gaussian parameters
print(f"\nLearned Parameters:")
print(f"μ (means): {mu}")
print(f"σ² (variances): {sigma2}")
print(f"ε (threshold): {epsilon:.2e}")

### 7. Multivariate Gaussian Anomaly Detection

For better performance, we can use the full multivariate Gaussian instead of assuming independent features.

In [ ]:
def multivariate_gaussian_anomaly_detection(X_train, X_val):
    """Multivariate Gaussian anomaly detection"""
    
    # Estimate parameters
    mu = np.mean(X_train, axis=0)
    cov = np.cov(X_train.T)
    
    # Create multivariate Gaussian
    rv = multivariate_normal(mu, cov)
    
    # Select threshold on validation set
    p_val = rv.pdf(X_val)
    y_val = np.zeros(X_val.shape[0])  # Assume mostly normal
    y_val[:int(0.1 * len(y_val))] = 1  # 10% anomalies for demo
    
    # Find best threshold
    best_epsilon = None
    best_f1 = 0
    
    for epsilon in np.logspace(-20, 0, 100):
        predictions = (p_val < epsilon).astype(int)
        
        tp = np.sum((predictions == 1) & (y_val == 1))
        fp = np.sum((predictions == 1) & (y_val == 0))
        fn = np.sum((predictions == 0) & (y_val == 1))
        
        if tp + fp > 0 and tp + fn > 0:
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1 = 2 * precision * recall / (precision + recall)
            
            if f1 > best_f1:
                best_f1 = f1
                best_epsilon = epsilon
    
    print(f"Multivariate Gaussian: Selected ε = {best_epsilon:.2e}, F1 = {best_f1:.4f}")
    
    return mu, cov, best_epsilon, rv

# Train multivariate model
mu_mv, cov_mv, epsilon_mv, rv_mv = multivariate_gaussian_anomaly_detection(X_train_anomaly, X_test_anomaly)

# Make predictions
p_test_mv = rv_mv.pdf(X_test_anomaly)
y_pred_mv = (p_test_mv < epsilon_mv).astype(int)

# Evaluate
print("\nMultivariate Gaussian Results:")
print(classification_report(y_test_anomaly, y_pred_mv, target_names=['Normal', 'Anomaly']))

# Visualize the Gaussian contours
plt.figure(figsize=(10, 8))

# Create grid for contour plot
x_min, x_max = X_test_anomaly[:, 0].min() - 1, X_test_anomaly[:, 0].max() + 1
y_min, y_max = X_test_anomaly[:, 1].min() - 1, X_test_anomaly[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                    np.linspace(y_min, y_max, 100))
grid_points = np.column_stack([xx.ravel(), yy.ravel()])

# Compute probabilities on grid
Z = rv_mv.pdf(grid_points).reshape(xx.shape)

# Plot contours
plt.contourf(xx, yy, Z, levels=20, cmap='Blues', alpha=0.6)
plt.colorbar(label='Probability Density')

# Plot decision boundary
plt.contour(xx, yy, Z, levels=[epsilon_mv], colors='red', linewidths=2, linestyles='--')

# Plot data points
plt.scatter(X_test_anomaly[y_test_anomaly == 0, 0], X_test_anomaly[y_test_anomaly == 0, 1], 
           c='blue', alpha=0.7, label='Normal')
plt.scatter(X_test_anomaly[y_test_anomaly == 1, 0], X_test_anomaly[y_test_anomaly == 1, 1], 
           c='red', alpha=0.7, label='Anomaly')

# Mark misclassifications
misclassified = y_pred_mv != y_test_anomaly
plt.scatter(X_test_anomaly[misclassified, 0], X_test_anomaly[misclassified, 1], 
           c='yellow', edgecolors='black', s=100, label='Misclassified')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Multivariate Gaussian Anomaly Detection')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')

plt.show()

# Compare covariance matrices
print(f"\nCovariance Matrix:\n{cov_mv}")
print(f"\nCorrelation: {cov_mv[0,1] / np.sqrt(cov_mv[0,0] * cov_mv[1,1]):.3f}")

### 8. Practical Applications and Considerations

Let's explore real-world applications and important considerations for unsupervised learning.

In [ ]:
# Customer segmentation using K-means
from sklearn.datasets import make_classification

# Simulate customer data
np.random.seed(42)
n_customers = 500

# Features: age, income, spending_score
customer_data = np.random.multivariate_normal(
    [35, 50000, 50], 
    [[100, 5000, 25], [5000, 1000000, 1000], [25, 1000, 100]], 
    n_customers
)

# Add some distinct customer segments
young_high_spenders = np.random.multivariate_normal(
    [25, 80000, 80], 
    [[25, 2000, 5], [2000, 400000, 500], [5, 500, 25]], 
    100
)

older_low_spenders = np.random.multivariate_normal(
    [60, 30000, 20], 
    [[100, 1000, 2], [1000, 100000, 100], [2, 100, 9]], 
    100
)

# Combine all customer data
all_customers = np.vstack([customer_data, young_high_spenders, older_low_spenders])

# Scale the data
scaler = StandardScaler()
customers_scaled = scaler.fit_transform(all_customers)

# Apply K-means clustering
kmeans_customers = KMeans(n_clusters=4, random_state=42, n_init=10)
customer_clusters = kmeans_customers.fit_predict(customers_scaled)

# Visualize customer segments (using first 3 features)
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(customers_scaled[:, 0], customers_scaled[:, 1], customers_scaled[:, 2], 
                    c=customer_clusters, cmap='viridis', alpha=0.7)

ax.set_xlabel('Age (scaled)')
ax.set_ylabel('Income (scaled)')
ax.set_zlabel('Spending Score (scaled)')
ax.set_title('Customer Segmentation using K-means')

plt.colorbar(scatter, label='Cluster')
plt.show()

# Analyze cluster characteristics
print("\nCustomer Segment Analysis:")
for cluster_id in range(4):
    cluster_data = all_customers[customer_clusters == cluster_id]
    print(f"\nCluster {cluster_id}: {len(cluster_data)} customers")
    print(f"  Average Age: {cluster_data[:, 0].mean():.1f}")
    print(f"  Average Income: ${cluster_data[:, 1].mean():,.0f}")
    print(f"  Average Spending Score: {cluster_data[:, 2].mean():.1f}")

# Evaluate clustering quality
silhouette_avg = silhouette_score(customers_scaled, customer_clusters)
print(f"\nClustering Quality:")
print(f"Silhouette Score: {silhouette_avg:.3f}")
print(f"Calinski-Harabasz Score: {calinski_harabasz_score(customers_scaled, customer_clusters):.2f}")

### 9. Experimentation and Questions

Now let's explore some advanced clustering and anomaly detection concepts:

1. **K-means Initialization**: Compare different initialization methods (random, k-means++, etc.)

2. **Distance Metrics**: Experiment with different distance measures for K-means

3. **Anomaly Detection Evaluation**: How do you evaluate anomaly detection when you don't have labeled anomalies?

4. **Scalability**: How do these algorithms scale with large datasets?

5. **Robustness**: How sensitive are these methods to outliers and noise?

**Challenge**: Implement a hierarchical clustering algorithm and compare it with K-means.

In [ ]:
# Experiment: K-means sensitivity to initialization
def compare_kmeans_initializations(X, K=4, n_runs=10):
    """Compare different K-means initialization methods"""
    
    methods = ['random', 'k-means++']
    results = {method: [] for method in methods}
    
    for method in methods:
        print(f"Testing {method} initialization...")
        for run in range(n_runs):
            kmeans = KMeans(n_clusters=K, init=method, n_init=1, random_state=run)
            kmeans.fit(X)
            results[method].append(kmeans.inertia_)
    
    return results

# Run initialization comparison
init_results = compare_kmeans_initializations(X_clusters)

# Plot results
plt.figure(figsize=(10, 6))

methods = list(init_results.keys())
data = [init_results[method] for method in methods]

plt.boxplot(data, labels=methods)
plt.ylabel('Final WCSS')
plt.title('K-means Initialization Methods Comparison')
plt.grid(True, alpha=0.3)

for i, method in enumerate(methods):
    plt.text(i+1, np.mean(init_results[method]) + 5, f'Mean: {np.mean(init_results[method]):.1f}',
             ha='center', va='bottom')

plt.show()

print("\nInitialization Statistics:")
for method in methods:
    wcss_values = init_results[method]
    print(f"{method}:")
    print(f"  Mean WCSS: {np.mean(wcss_values):.2f}")
    print(f"  Std WCSS: {np.std(wcss_values):.2f}")
    print(f"  Best WCSS: {np.min(wcss_values):.2f}")
    print(f"  Worst WCSS: {np.max(wcss_values):.2f}")

# Demonstrate the importance of good initialization
print("\nNote: K-means++ typically gives more consistent and better results than random initialization.")
print("This is especially important when clusters have similar sizes or are close together.")

### Key Takeaways

1. **K-means Clustering** partitions data into K clusters by minimizing within-cluster distances.

2. **Choosing K** is crucial and can be done using elbow method, silhouette analysis, or domain knowledge.

3. **Anomaly Detection** identifies unusual data points using statistical modeling (Gaussian approach).

4. **Multivariate Gaussian** accounts for feature correlations and often performs better than univariate approach.

5. **Initialization Matters** in K-means - K-means++ initialization is generally superior to random initialization.

### Next Steps

In the next notebook, we'll explore recommender systems and collaborative filtering, learning how to build personalized recommendation engines that work with user-item interaction data.